In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import os 

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences

In [2]:
def get_times_series_data(df, cut_date, time_steps=3):
    columns = list(df.columns.drop(["R28D"]))
    columns.append("R28D")
    df = df.reindex(columns=columns)
    df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
    df_train = df[df["Data/Hora"] < cut_date]
    df_test = df[df["Data/Hora"] >= cut_date]
    df_train = df_train.drop(["Data/Hora"], axis=1)
    df_test = df_test.drop(["Data/Hora"], axis=1)
    x_train, y_train = split_sequences(df_train.values, time_steps)
    x_test, y_test = split_sequences(df_test.values, time_steps)
    return x_train, y_train, x_test, y_test

In [3]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [4]:
def get_baseline_model(n_units=64, learning_rate=0.01):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.LSTM(units=n_units))
    model.add(tf.keras.layers.Dense(1))

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [5]:
def create_keras_model(
    activation="tanh", dropout_rate=0.0, n_units=64, n_layers=1, learning_rate=1e-3
):
    model = tf.keras.Sequential()

    for i in range(n_layers):
        if i + 1 < n_layers:
            model.add(
                tf.keras.layers.LSTM(
                    units=n_units, activation=activation, return_sequences=True
                )
            )
        else:
            model.add(
                tf.keras.layers.LSTM(
                    units=n_units, activation=activation, return_sequences=False
                )
            )
    if dropout_rate > 0.0:
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [6]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    grid=None,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
    params=None,
):
    results = []
    max_samples = x.shape[0]

    if grid:
        activation = grid.best_params_["model__activation"]
        dropout_rate = grid.best_params_["model__dropout_rate"]
        n_units = grid.best_params_["model__n_units"]
        n_layers = grid.best_params_["model__n_layers"]
        learning_rate = grid.best_params_["model__learning_rate"]
        batch_size = grid.best_params_["model__batch_size"]
        epochs = grid.best_params_["model__epochs"]
    else:
        activation = params["activation"]
        dropout_rate = params["dropout_rate"]
        n_units = params["n_units"]
        n_layers = params["n_layers"]
        learning_rate = params["learning_rate"]
        batch_size = params["batch_size"]
        epochs = params["epochs"]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = create_keras_model(
                activation=activation,
                dropout_rate=dropout_rate,
                n_units=n_units,
                n_layers=n_layers,
                learning_rate=learning_rate,
            )
            model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results


In [7]:
def make_timesteps_repeated_time_series_k_fold(
    df,
    train_period,
    test_period,
    params=None,
    grid=None,
    repeats=10,
    timesteps_list=[3],
    show_results=True,
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x, y, train_period, test_period, grid, repeats, False, False, params
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results


<h3>Disable Tensorflow Warnings</h3>

In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [9]:
df_r3d_and_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_and_r7d-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_r3d_only = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_only-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_no_r3d_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "no-r3d-r7d-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)

## Feature Selection

### In this notebook we remove variables that could potentially cause data leakage. Thus we are performing testings removing the following variables:

#### IP - Initial setting time
#### FP - Final setting time


#### BL - Blaine specific surface (This is both removed and use)

In [10]:
seed = 47
scaler = StandardScaler()

# Neural Network Long Short Term Memory - LSTM

# Configs:

1. Train Period: [01-01-2019, 2021-05-02)

2. Test Period:  (2021-05-02, 2021-11-31]

3. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

4. Epochs: 100

5. Batch size: 32

6. LSTM Cells: 64

7. LSTM Layers: 1

8. Optimization Algorithm: Adam
    
    8.1. Learning Rate: 1e-2
    
<h3>Repeats: 10</h3>
<h3>K: 10</h3>

# 1. Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation with all 3 datasets. Here we are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


In [11]:
params = {}
params["activation"] = "tanh"
params["dropout_rate"] = 0.0
params["n_units"] = 64
params["n_layers"] = 1
params["learning_rate"] = 1e-2
params["batch_size"] = 32
params["epochs"] = 100

timesteps_list = [1, 3, 5, 7, 10, 15, 20]

repeats = 10

## 1.1 Dataset: no-r3d-r7d-no-resampling-with-fillna

<h3>1.1.1 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [12]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [13]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [14]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 8.948 (3.416)
TIMESTEPS: 3 RMSE: 2.006 (0.557)
TIMESTEPS: 5 RMSE: 1.693 (0.454)
TIMESTEPS: 7 RMSE: 1.686 (0.483)
TIMESTEPS: 10 RMSE: 1.745 (0.480)
TIMESTEPS: 15 RMSE: 1.816 (0.292)
TIMESTEPS: 20 RMSE: 2.642 (2.290)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [15]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [16]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 6.091 (1.030)
TIMESTEPS: 3 RMSE: 2.277 (0.931)
TIMESTEPS: 5 RMSE: 4.148 (7.014)
TIMESTEPS: 7 RMSE: 2.003 (0.955)
TIMESTEPS: 10 RMSE: 1.899 (0.774)
TIMESTEPS: 15 RMSE: 2.824 (0.108)
TIMESTEPS: 20 RMSE: 2.218 (0.074)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [17]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [18]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 5.062 (0.709)
TIMESTEPS: 3 RMSE: 2.680 (0.049)
TIMESTEPS: 5 RMSE: 3.386 (2.130)
TIMESTEPS: 7 RMSE: 2.611 (0.039)
TIMESTEPS: 10 RMSE: 2.533 (0.004)
TIMESTEPS: 15 RMSE: 2.362 (0.007)
TIMESTEPS: 20 RMSE: 2.286 (0.003)


<h3>1.1.2 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP, FP and BL variables are discarded.

In [19]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [20]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [21]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 8.257 (3.062)
TIMESTEPS: 3 RMSE: 1.994 (0.540)
TIMESTEPS: 5 RMSE: 1.689 (0.453)
TIMESTEPS: 7 RMSE: 1.862 (0.999)
TIMESTEPS: 10 RMSE: 1.738 (0.484)
TIMESTEPS: 15 RMSE: 2.185 (2.044)
TIMESTEPS: 20 RMSE: 2.204 (0.297)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [22]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [23]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.684 (0.841)
TIMESTEPS: 3 RMSE: 1.825 (0.431)
TIMESTEPS: 5 RMSE: 1.994 (0.832)
TIMESTEPS: 7 RMSE: 2.338 (1.342)
TIMESTEPS: 10 RMSE: 1.881 (0.771)
TIMESTEPS: 15 RMSE: 2.804 (0.105)
TIMESTEPS: 20 RMSE: 2.261 (0.053)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [24]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [25]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 5.091 (0.652)
TIMESTEPS: 3 RMSE: 2.647 (0.041)
TIMESTEPS: 5 RMSE: 2.688 (0.025)
TIMESTEPS: 7 RMSE: 2.579 (0.031)
TIMESTEPS: 10 RMSE: 3.315 (2.340)
TIMESTEPS: 15 RMSE: 2.367 (0.012)
TIMESTEPS: 20 RMSE: 2.287 (0.003)


<h3>1.1.3 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> 

1. The IP, FP and BL variables are discarded.

2. High correlated variables are removed


In [26]:
to_drop = [
    "Data/Hora",
    "IP",
    "FP",
    "BL",
    "CALCITA",
    "DESIDRATAÇÃO",
    "GIPSITA",
    "TIO2",
    "AL2O3",
    "PF",
    "FERRITA",
]

df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(to_drop, axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [27]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [28]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 9.222 (4.166)
TIMESTEPS: 3 RMSE: 1.979 (0.553)
TIMESTEPS: 5 RMSE: 1.688 (0.451)
TIMESTEPS: 7 RMSE: 2.101 (2.226)
TIMESTEPS: 10 RMSE: 1.740 (0.486)
TIMESTEPS: 15 RMSE: 1.814 (0.290)
TIMESTEPS: 20 RMSE: 3.421 (4.712)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [29]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [30]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.249 (0.704)
TIMESTEPS: 3 RMSE: 1.778 (0.549)
TIMESTEPS: 5 RMSE: 2.057 (0.902)
TIMESTEPS: 7 RMSE: 2.099 (1.031)
TIMESTEPS: 10 RMSE: 1.885 (0.774)
TIMESTEPS: 15 RMSE: 2.827 (0.151)
TIMESTEPS: 20 RMSE: 2.249 (0.062)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [31]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [32]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 6.515 (1.119)
TIMESTEPS: 3 RMSE: 2.622 (0.067)
TIMESTEPS: 5 RMSE: 2.697 (0.029)
TIMESTEPS: 7 RMSE: 2.581 (0.018)
TIMESTEPS: 10 RMSE: 2.537 (0.018)
TIMESTEPS: 15 RMSE: 2.363 (0.005)
TIMESTEPS: 20 RMSE: 2.296 (0.014)


## 1.2 Dataset: df_r3d_only

<h3>1.2.1 Time Series Repeated KFold Cross Validation - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset only R3D variable is considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [33]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [34]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [35]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 10.226 (2.857)
TIMESTEPS: 3 RMSE: 1.643 (0.428)
TIMESTEPS: 5 RMSE: 1.368 (0.154)
TIMESTEPS: 7 RMSE: 1.344 (0.090)
TIMESTEPS: 10 RMSE: 1.428 (0.205)
TIMESTEPS: 15 RMSE: 1.659 (0.118)
TIMESTEPS: 20 RMSE: 1.806 (0.344)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [36]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [37]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 3.577 (0.400)
TIMESTEPS: 3 RMSE: 2.506 (0.074)
TIMESTEPS: 5 RMSE: 2.409 (0.036)
TIMESTEPS: 7 RMSE: 2.299 (0.146)
TIMESTEPS: 10 RMSE: 2.232 (0.014)
TIMESTEPS: 15 RMSE: 1.753 (0.065)
TIMESTEPS: 20 RMSE: 1.395 (0.034)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [38]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [39]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 2.817 (0.397)
TIMESTEPS: 3 RMSE: 2.421 (0.253)
TIMESTEPS: 5 RMSE: 1.244 (0.122)
TIMESTEPS: 7 RMSE: 1.247 (0.015)
TIMESTEPS: 10 RMSE: 1.296 (0.001)
TIMESTEPS: 15 RMSE: 1.467 (0.062)
TIMESTEPS: 20 RMSE: 1.609 (0.056)


<h3>1.2.2 Time Series Repeated KFold Cross Validation with vary timesteps - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP, FP and BL variables are discarded.

In [40]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [41]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [42]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 11.279 (2.873)
TIMESTEPS: 3 RMSE: 1.642 (0.393)
TIMESTEPS: 5 RMSE: 1.375 (0.148)
TIMESTEPS: 7 RMSE: 1.344 (0.092)
TIMESTEPS: 10 RMSE: 1.423 (0.202)
TIMESTEPS: 15 RMSE: 2.337 (2.872)
TIMESTEPS: 20 RMSE: 1.778 (0.284)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [43]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [44]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.274 (0.477)
TIMESTEPS: 3 RMSE: 2.509 (0.073)
TIMESTEPS: 5 RMSE: 2.409 (0.037)
TIMESTEPS: 7 RMSE: 2.286 (0.045)
TIMESTEPS: 10 RMSE: 2.231 (0.010)
TIMESTEPS: 15 RMSE: 1.764 (0.048)
TIMESTEPS: 20 RMSE: 1.398 (0.020)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [45]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [46]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 2.943 (0.360)
TIMESTEPS: 3 RMSE: 2.609 (0.270)
TIMESTEPS: 5 RMSE: 1.206 (0.031)
TIMESTEPS: 7 RMSE: 1.243 (0.035)
TIMESTEPS: 10 RMSE: 1.286 (0.035)
TIMESTEPS: 15 RMSE: 1.494 (0.023)
TIMESTEPS: 20 RMSE: 2.908 (3.778)


<h3>1.2.3 Time Series Repeated KFold Cross Validation with vary timesteps - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b>

1. The IP, FP and BL variables are discarded.

2. High correlated variables are removed


In [47]:
to_drop = [
    "Data/Hora",
    "IP",
    "FP",
    "BL",
    "CALCITA",
    "DESIDRATAÇÃO",
    "GIPSITA",
    "TIO2",
    "AL2O3",
    "PF",
    "FERRITA",
]
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(to_drop, axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [48]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [49]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 10.035 (1.924)
TIMESTEPS: 3 RMSE: 1.668 (0.400)
TIMESTEPS: 5 RMSE: 1.362 (0.133)
TIMESTEPS: 7 RMSE: 1.346 (0.096)
TIMESTEPS: 10 RMSE: 1.420 (0.200)
TIMESTEPS: 15 RMSE: 1.670 (0.125)
TIMESTEPS: 20 RMSE: 1.774 (0.278)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [50]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [51]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.459 (0.420)
TIMESTEPS: 3 RMSE: 2.485 (0.077)
TIMESTEPS: 5 RMSE: 2.365 (0.091)
TIMESTEPS: 7 RMSE: 2.306 (0.022)
TIMESTEPS: 10 RMSE: 2.229 (0.042)
TIMESTEPS: 15 RMSE: 1.773 (0.052)
TIMESTEPS: 20 RMSE: 1.373 (0.048)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [52]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [53]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 3.536 (0.438)
TIMESTEPS: 3 RMSE: 2.261 (0.304)
TIMESTEPS: 5 RMSE: 1.187 (0.008)
TIMESTEPS: 7 RMSE: 1.231 (0.037)
TIMESTEPS: 10 RMSE: 1.294 (0.005)
TIMESTEPS: 15 RMSE: 1.495 (0.028)
TIMESTEPS: 20 RMSE: 1.648 (0.012)


## 1.3 Dataset: df_r3d_and_r7d

<h3>1.3.1 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [54]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [55]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [56]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 10.611 (3.724)
TIMESTEPS: 3 RMSE: 1.593 (0.354)
TIMESTEPS: 5 RMSE: 1.305 (0.144)
TIMESTEPS: 7 RMSE: 1.399 (0.186)
TIMESTEPS: 10 RMSE: 1.542 (0.146)
TIMESTEPS: 15 RMSE: 1.621 (0.178)
TIMESTEPS: 20 RMSE: 1.795 (0.262)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [57]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [58]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.127 (0.199)
TIMESTEPS: 3 RMSE: 2.283 (0.112)
TIMESTEPS: 5 RMSE: 2.326 (0.058)
TIMESTEPS: 7 RMSE: 2.261 (0.050)
TIMESTEPS: 10 RMSE: 2.223 (0.115)
TIMESTEPS: 15 RMSE: 1.749 (0.045)
TIMESTEPS: 20 RMSE: 1.219 (0.139)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [59]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [60]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 2.983 (0.375)
TIMESTEPS: 3 RMSE: 2.077 (0.156)
TIMESTEPS: 5 RMSE: 1.263 (0.053)
TIMESTEPS: 7 RMSE: 1.202 (0.005)
TIMESTEPS: 10 RMSE: 1.339 (0.071)
TIMESTEPS: 15 RMSE: 1.507 (0.086)
TIMESTEPS: 20 RMSE: 1.714 (0.140)


<h3>1.3.2 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b> The IP, FP, and BL variables are discarded.

In [61]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [62]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [63]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 11.014 (2.684)
TIMESTEPS: 3 RMSE: 1.588 (0.335)
TIMESTEPS: 5 RMSE: 1.302 (0.132)
TIMESTEPS: 7 RMSE: 2.470 (5.738)
TIMESTEPS: 10 RMSE: 2.513 (4.155)
TIMESTEPS: 15 RMSE: 1.616 (0.174)
TIMESTEPS: 20 RMSE: 1.795 (0.262)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [64]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [65]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.622 (0.333)
TIMESTEPS: 3 RMSE: 2.260 (0.096)
TIMESTEPS: 5 RMSE: 2.355 (0.035)
TIMESTEPS: 7 RMSE: 2.240 (0.013)
TIMESTEPS: 10 RMSE: 2.220 (0.022)
TIMESTEPS: 15 RMSE: 1.767 (0.036)
TIMESTEPS: 20 RMSE: 1.269 (0.115)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [66]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [67]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 3.047 (0.358)
TIMESTEPS: 3 RMSE: 2.009 (0.209)
TIMESTEPS: 5 RMSE: 1.287 (0.057)
TIMESTEPS: 7 RMSE: 2.264 (3.229)
TIMESTEPS: 10 RMSE: 1.355 (0.005)
TIMESTEPS: 15 RMSE: 1.539 (0.006)
TIMESTEPS: 20 RMSE: 1.799 (0.023)


<h3>1.3.3 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b>
    1. The IP, FP and BL variables are discarded.
    2. High correlated variables are removed


In [68]:
to_drop = [
    "Data/Hora",
    "IP",
    "FP",
    "BL",
    "CALCITA",
    "DESIDRATAÇÃO",
    "GIPSITA",
    "TIO2",
    "AL2O3",
    "PF",
    "FERRITA",
]

df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(to_drop, axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [69]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [70]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 10.375 (1.655)
TIMESTEPS: 3 RMSE: 1.542 (0.303)
TIMESTEPS: 5 RMSE: 1.750 (2.498)
TIMESTEPS: 7 RMSE: 3.847 (9.112)
TIMESTEPS: 10 RMSE: 3.256 (5.312)
TIMESTEPS: 15 RMSE: 2.619 (4.315)
TIMESTEPS: 20 RMSE: 4.965 (10.777)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [71]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [72]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 4.615 (0.386)
TIMESTEPS: 3 RMSE: 2.328 (0.108)
TIMESTEPS: 5 RMSE: 2.310 (0.059)
TIMESTEPS: 7 RMSE: 2.251 (0.044)
TIMESTEPS: 10 RMSE: 2.181 (0.119)
TIMESTEPS: 15 RMSE: 1.752 (0.067)
TIMESTEPS: 20 RMSE: 1.254 (0.067)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [73]:
O que antes era algo com que eu sonhava tanto, idealizava - tocar, ganhar dinheiro com música, ter um trabalho artísticotrain_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [74]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 3.168 (0.402)
TIMESTEPS: 3 RMSE: 2.143 (0.283)
TIMESTEPS: 5 RMSE: 1.258 (0.008)
TIMESTEPS: 7 RMSE: 1.197 (0.007)
TIMESTEPS: 10 RMSE: 1.353 (0.044)
TIMESTEPS: 15 RMSE: 1.513 (0.082)
TIMESTEPS: 20 RMSE: 1.795 (0.022)
